# 1-D

In [ ]:
import os
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir(os.pardir)

import matplotlib.pyplot as plt
import numpy as np

from pycbem.bhte import init_temp, delta_temp_analytic, delta_temp
from pycbem.constants import eps_0, mu_0
from pycbem.utils.dataloader import load_tissue_diel_properties
from pycbem.utils.viz import fig_config

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
## constants
# dry skin bhte parameters from IT'IS database
# https://itis.swiss/virtual-population/tissue-properties/database/
k = 0.37       # thermal conductivity [W/m/°C]
rho = 1109.    # dry skin density [kg/m^3]
C = 3391.      # specific heat of dry skin [Ws/kg/°C]
m_b = 1.76e-6  # blood perfusion [m^3/kg/s], equivalent to 106 mL/min/kg

# for blood
k_b = 0.52     # thermal conductivity [W/m/°C]
rho_b = 1000.  # kg/m^3
C_b = 3617.    # specific heat of blood [J/kg/°C]

# additional parameters
h_0 = 10.      # heat convection coefficient skin surface - air [W/m^2/°C]
T_a = 37.      # arterial temperature [°C]     
T_c = 37.      # body core temperature [°C]
T_f = 25.      # surrounding air temperature [°C]
Q_m = 33800.   # metabolic heat generation [W/m^3]

## 1-D bioheat transfer equation in time

Comparisson of the spectral method time domain solution and the analytic solution proposed in Foster et al. 2016.\
Initial temperature distribution is obtained via an analytic expression proposed in Deng et al. 2002.

### Setup

In [ ]:
# frequency
f = 6e9

# conductivity, relative permitivity, tangent loss and penetration depth
# from Gabriel et al. 1996
sigma, eps_r, tan_loss, pen_depth = load_tissue_diel_properties('skin_dry', f)

# `pen_depth` is the energy penetration depth into tissue, which is defined as
# the distance beneath the surface at which the SAR has fallen to a factor of
# 1/e below that at the surface; one-half of the more commonly reported wave
# penetration depth
pen_depth = pen_depth / 2

# air (vacuum) resistance 
Z_air = np.sqrt(mu_0 / eps_0)

# dry skin resistance
Z_skin_dry = np.sqrt(mu_0 / (eps_r * eps_0))

# energy (Fresnel) transmission coefficient into the tissue
T_tr = 2 * Z_skin_dry / (Z_air + Z_skin_dry)

# I0 T_Tr = 10 W/m^2 at 6 GHz
I0 = 10. / T_tr

### Simulation

In [ ]:
# exposure time in seconds
sim_time = 360
t = np.linspace(0, sim_time, 101)

# number of collocation points for domain discretization
N = 101

# run
deltaT = delta_temp(t, N, pen_depth, k, rho, C, m_b, I0, T_tr)

### Visualizations

Temperature change on dry skin surface during a constant mmWave exposure.

In [ ]:
fig_config(scaler=1.25)
fig = plt.figure()
ax = fig.add_subplot()
ax.plot(t, delta_temp_analytic(t, pen_depth, k, rho, C, I0, T_tr), 'k--',
        label='$\\Delta T(t, z=0)$, Foster et al.')
ax.plot(t, deltaT[:, 0], 'k-', label='$\\Delta T(t, z=0)$, 1-D PSTD')
ax.set_xlabel('$t$ [s]')
ax.set_ylabel('$\\Delta T$ [°C]')
ax.legend()
ax.grid()
plt.tight_layout()
plt.show()

Temperature evolution on the surface and the distribution of temperature before (Deng2002) and after the exposure.

In [ ]:
dx = pen_depth / N
z = np.linspace(0, pen_depth, N)
T0 = init_temp(z, k, rho, C, rho_b, C_b, m_b, h_0, T_a, T_c, T_f, Q_m)
T = T0 + deltaT

fig_config(nrows=1, ncols=2, scaler=1.25)
fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(t, np.ones_like(t) * T0[0], 'k--', label='$T(z=0)$, Deng et al.')
ax1.plot(t, T[:, 0], 'k-', label='$T(t, z=0)$, 1-D PSTD')
ax1.set_xlabel('$t$ [s]')
ax1.set_ylabel('$T$ [°C]')
ax1.legend()
ax1.grid()

ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(z * 1e3, T0, 'k--',label='$T(z)$, Deng et al.')
ax2.plot(z * 1e3, T[-1, :], 'k-', label=f'$T(t={int(t[-1])}, z)$, 1-D PSTD')
ax2.set_xlabel('$z$ [mm]')
ax2.set_ylabel('$T$ [°C]')
ax2.legend()
ax2.grid()
plt.tight_layout()
plt.show()

### Proper orthogonal decomposition of the system dynamics

In [ ]:
U, S, VT = np.linalg.svd(deltaT.T)

# the most significant orthogonal modes in the model
fig_config(nrows=1, ncols=2, scaler=1.25)
fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1)
ax1.semilogy(100 * S / np.sum(S), 'k.')
ax1.semilogy(0, 100 * S[0] / np.sum(S), 'ro')
ax1.semilogy(1, 100 * S[1] / np.sum(S), 'go')
ax1.semilogy(2, 100 * S[2] / np.sum(S), 'bo')
ax1.set_xlabel('k')
ax1.set_ylabel('$\\sigma_k$')
ax1.grid()

n_modes = 3
color = ['r', 'g', 'b']
linewidth = [2.5, 2, 1.5]
alpha = [1, 0.9, 0.8]
ax2 = fig.add_subplot(1, 2, 2)
for idx in range(n_modes):
    ax2.plot(z * 1e3, np.real(U[:, idx]), c=color[idx],
             linewidth=linewidth[idx], alpha=alpha[idx], label=f'{idx + 1}')
ax2.set_ylim(1.1 * np.min(U[:, 1].real), 1.1 * np.max(U[:, 1].real))
ax2.set_xlabel('tissue depth [mm]')
ax2.set_ylabel('$\\Psi_k$')
ax2.grid()
ax2.legend(loc='lower right')

plt.tight_layout()
plt.show()